In [1]:
from utils import filter_img_paths, expand_box, read_pts
from data import Dataset, make_loader
from models import BaselineModel
from metrics import calc_ced, count_ced_auc
from config import DATA_DIR, LOG_DIR, N_EPOCHS, BATCH_SIZE

%load_ext autoreload
%autoreload 2

In [2]:
import os.path
from pathlib import Path

TRAIN_PATH_300W = os.path.join(DATA_DIR, '300W/train')
TRAIN_PATH_MENPO = os.path.join(DATA_DIR, 'Menpo/train')

TEST_PATH_300W = os.path.join(DATA_DIR, '300W/test')
TEST_PATH_MENPO = os.path.join(DATA_DIR, 'Menpo/test')

In [7]:
img_paths = filter_img_paths([Path(TRAIN_PATH_300W).glob('*.jpg'),
              Path(TRAIN_PATH_300W).glob('*.png'),
              Path(TRAIN_PATH_MENPO).glob('*.jpg')])

In [8]:
test_paths = filter_img_paths([Path(TEST_PATH_MENPO).glob('*.jpg')], check_box=True)

In [9]:
from sklearn.model_selection import train_test_split

train_paths, val_paths = train_test_split(img_paths, test_size=0.2, random_state=0)

In [10]:
train_ds, train_dl = make_loader(train_paths, mode='train', batch_size=BATCH_SIZE, num_workers=0, k=0.12)
val_ds, val_dl = make_loader(val_paths, mode='val', batch_size=BATCH_SIZE, num_workers=0, k=0.12)
test_ds, test_dl = make_loader(test_paths, mode='test', batch_size=BATCH_SIZE, num_workers=0, k=0.12)

In [11]:
from runner import CustomRunner

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [62]:
import torch
import random
import numpy as np
from catalyst.utils import set_global_seed

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True
set_global_seed(0)

In [63]:
model = BaselineModel()

In [64]:
from torch import nn, optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

runner = CustomRunner()

criterion = nn.MSELoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=0.01)
scheduler = ReduceLROnPlateau(optimizer, factor=0.5, patience=2)

In [ ]:
runner.train(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    loaders={
        "train": train_dl,
        "valid": val_dl,
        'infer': test_dl
    },
    num_epochs=N_EPOCHS,
    logdir=LOG_DIR,
    valid_loader="valid",
    valid_metric="loss",
    minimize_valid_metric=True,
    verbose=True,
    # fp16=True,
    # check=True
)